In [ ]:
import re
import os
import csv
import time
import shutil
import PyPDF2
import pandas as pd
from docx import Document
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
Registers = []
with open("C:/Analytics/Lista_de_Ruc.csv", mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        Registers.append(row[0])  # Suponiendo que el dato está en la primera columna
Registers

['1790300404001',
 '1391738986001',
 '1391880081001',
 "0891742622001'",
 '1391932105001',
 '1391797486001',
 '1391898800001',
 '1390012949001',
 '1391877986001',
 '1391932601001',
 '1391794495001',
 '1391932573001',
 '1391800673001',
 "0990604282001'",
 '1391822189001',
 '1391815123001',
 '1391933738001']

In [ ]:
driver = webdriver.Chrome()
#driver.maximize_window()
wait = WebDriverWait(driver, 10)
data = []
fecha_consulta = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
for register in Registers:
    driver.get("https://www.iess.gob.ec/empleador-web/pages/morapatronal/certificadoCumplimientoPublico.jsf")
    time.sleep(2)
    input_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/form/table/tbody/tr/td/table/tbody/tr[2]/td/div[1]/table/tbody/tr[2]/td/input")))
    # Ingresar el número en el campo de texto
    input_element.send_keys(register)
    time.sleep(3)
    #Click en boton consultar
    elemento_xpath = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/form/table/tbody/tr/td/table/tbody/tr[2]/td/div[1]/div[5]/input")))
    elemento_xpath.click()
    time.sleep(3)  
    
    #Controla si sale error al ingresar el ruc ingresa No aplica en el excel 
    try:
        error_message = driver.find_element(By.CLASS_NAME, "rich-messages-label")
        if error_message.is_displayed():
            print(f"RUC {register} es incorrecto. No aplica Iess")
            data.append([fecha_consulta, register, "No aplica", "No aplica"])
            continue  # Salta al siguiente RUC sin procesar PDF
    except:

        pass
    
driver.quit()

#rutas de archivos
downloads_folder = os.path.expanduser("C:/Users/eeras/Downloads")  # Ajusta TU_USUARIO
destination_folder = "C:/Analytics/Iess/pdf"
excel_path = "C:/Analytics/Iess/obligaciones_patronales.xlsx"


#mover archivos de descargas a la carpeta analytics
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

pattern = re.compile(r"certificado_empresa_ruc(?: \(\d+\))?\.pdf$")

secuencial = 1

#Mover y renombrar archivos PDF antes de procesar el contenido
for file in os.listdir(downloads_folder):
    if file.endswith(".pdf") and pattern.match(file):  
        pdf_path = os.path.join(downloads_folder, file)

        with open(pdf_path, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"

        ruc_match = re.search(r'Nro. ?\s*(\d{13})', text)
        # Si encontramos el RUC en el texto
        if ruc_match:
            ruc = ruc_match.group(1)
            new_filename = f"{ruc}_{secuencial}.pdf"  
            secuencial += 1 
        else:
            new_filename = file 
            secuencial += 1 
        
        dest_path = os.path.join(destination_folder, new_filename)
 
        shutil.move(pdf_path, dest_path)
        print(f"Movido y renombrado: {file} -> {new_filename}")

        #Extracción de información
        with open(dest_path, "rb") as pdf_new_file_name:
            reader = PyPDF2.PdfReader(pdf_new_file_name)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
        
        ruc_match = re.search(r'Nro. ?\s*(\d{13})', text)
        razon_social_match = re.search(r'representante legal de la empresa\s*([A-Za-z0-9 ñÑáéíóúÁÉÍÓÚ&]+(?:\s+[A-Za-z0-9 ñÑáéíóúÁÉÍÓÚ&]+)*)', text)
        obligaciones_match = re.search(r'(NO\s*\n*registra|SI\s*\n*registra)(?:\s*|\n*)', text, re.IGNORECASE)

        ruc = ruc_match.group(1) if ruc_match else "No encontrado"

        #Razon social
        if razon_social_match:
            razon_social = razon_social_match.group(1).strip()
        else:
            razon_social = "No encontrado"

        #obligaciones
        if obligaciones_match:
            obligaciones = obligaciones_match.group(1).strip()
        else:
            obligaciones = "No encontrado"
            
        

        data.append([fecha_consulta, ruc, razon_social, obligaciones])

#Guardar en un archivo Excel
df = pd.DataFrame(data, columns=[ "Fecha de Consulta","RUC", "Razón Social", "Obligaciones Patronales"])
df.to_excel(excel_path, index=False)

print("Proceso completado. Información guardada")

RUC 1391932105001 es incorrecto. No aplica Iess
RUC 1391797486001 es incorrecto. No aplica Iess
RUC 1391898800001 es incorrecto. No aplica Iess
RUC 1391877986001 es incorrecto. No aplica Iess
Movido y renombrado: certificado_empresa_ruc (1).pdf -> 1391738986001_1.pdf
Movido y renombrado: certificado_empresa_ruc (10).pdf -> 1391822189001_2.pdf
Movido y renombrado: certificado_empresa_ruc (11).pdf -> 1391815123001_3.pdf
Movido y renombrado: certificado_empresa_ruc (12).pdf -> 1391933738001_4.pdf
Movido y renombrado: certificado_empresa_ruc (2).pdf -> 1391880081001_5.pdf
Movido y renombrado: certificado_empresa_ruc (3).pdf -> 0891742622001_6.pdf
Movido y renombrado: certificado_empresa_ruc (4).pdf -> 1390012949001_7.pdf
Movido y renombrado: certificado_empresa_ruc (5).pdf -> 1391932601001_8.pdf
Movido y renombrado: certificado_empresa_ruc (6).pdf -> 1391794495001_9.pdf
Movido y renombrado: certificado_empresa_ruc (7).pdf -> 1391932573001_10.pdf
Movido y renombrado: certificado_empresa_ruc